In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\niceq/.surprise_data/ml-100k


In [4]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [5]:
predictions = algo.test( testset )
print('prediction type :', type(predictions), 'size: ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size:  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [6]:
# 3개의 Prediction 객체에서 uid, iid, est 속성 추출
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [7]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [8]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

In [1]:
import pandas as pd

ratings = pd.read_csv("C:/Users/niceq/Documents/DataScience/Python ML Guide/Data/09. ml-latest-small/ratings.csv")
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv("C:/Users/niceq/Documents/DataScience/Python ML Guide/Data/09. ml-latest-small/ratings_noh.csv", index=False, header=False)

In [6]:
from surprise import Dataset
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file("C:/Users/niceq/Documents/DataScience/Python ML Guide/Data/09. ml-latest-small/ratings_noh.csv", reader=reader)

In [7]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [8]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv("C:/Users/niceq/Documents/DataScience/Python ML Guide/Data/09. ml-latest-small/ratings.csv")
reader = Reader(rating_scale=(0.5, 5))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [10]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame 에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv("C:/Users/niceq/Documents/DataScience/Python ML Guide/Data/09. ml-latest-small/ratings.csv")
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8715  0.8777  0.8730  0.8641  0.8708  0.8714  0.0044  
MAE (testset)     0.6708  0.6754  0.6690  0.6654  0.6677  0.6697  0.0033  
Fit time          0.90    0.86    0.86    0.89    1.04    0.91    0.07    
Test time         0.09    0.09    0.09    0.10    0.11    0.10    0.01    


{'test_rmse': array([0.87146169, 0.87772719, 0.87301569, 0.8640863 , 0.87082052]),
 'test_mae': array([0.67076088, 0.67535892, 0.66903807, 0.66544235, 0.66772312]),
 'fit_time': (0.9049160480499268,
  0.8641841411590576,
  0.8562514781951904,
  0.8935606479644775,
  1.044891119003296),
 'test_time': (0.09137654304504395,
  0.09197711944580078,
  0.08747029304504395,
  0.1004798412322998,
  0.10979580879211426)}

In [11]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mse'], cv=3)
gs.fit(data)

# 최고 RMSE Evluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8777674348590364
{'n_epochs': 20, 'n_factors': 50}


In [13]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds("C:/Users/niceq/Documents/DataScience/Python ML Guide/Data/09. ml-latest-small/ratings_noh.csv", reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [14]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [16]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv("C:/Users/niceq/Documents/DataScience/Python ML Guide/Data/09. ml-latest-small/movies.csv")

userId = 9
movieId = 42

# userId=9의 movieId 추출 (42 데이터 있는지 확인)
movieIds = ratings[ratings['userId']==userId]['movieId']
if movieIds[movieIds==movieId].count() == 0:
    print('사용자 아이디 {0}는 영화 아이디 {1}의 평점 없음'.format(userId, movieId))
print(movies[movies['movieId']==movieId])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [17]:
userId = 9
movieId = 42

uid = str(userId)
iid = str(movieId)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [18]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    # 모든 영화의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, userId)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [22]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
    # [Prediction(uid='userId', iid='1', est=3.69), Prediction(uid='userId', iid='2', est=2.98)...]

    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est

    # sortkey_est()반환 값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions=predictions[:top_n]

    # top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, userId)
top_movie_preds = recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10)

print('### Top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
    print(top_movie[1], ":" , top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
### Top-10 추천 영화 리스트 ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
